# 3. Protein-Ligand Unbinding

## Introduction

Besides binding thermodynamics, binding kinetics is an important aspect of host-guest complexation. Ideal ligands do not only have a favourable binding free energy, but it has a considerable residence time, which implies it has a high enough reaction barrier to prevent frequent binding events.

Unlike thermodynamics, studying kinetics require knowledge of the entire path of the process. In this section, we present an adaptive biasing algorithm, designed to acquire unbinding paths of protein-ligand complexes.



<center>
<img src="static/unbinding_chart.jpeg" alt="flowchart" width="600"/>
</center>

Details of the algorithm are available in [this](https://pubs.acs.org/doi/full/10.1021/acs.jctc.1c00924) publication, the code is available [online](https://github.com/rostaresearch/unbinding).

This tutorial contains a modified and simplified version to showcase its logic in this notebook.

### Notes about the unbinding trajectory
This path was run with certain settings, then the trajectories were stripped of ions and water molecules for this demonstrative analysis. By reanalysing a given iteration, this obviously does not change the way it was once run.

The example system is a trypsin-benzamidine complex, PDB ID [3ATL](https://www.rcsb.org/structure/3ATL). We used CHARMM36m for the protein, TIP3 water, and standard CGenFF parametrisation of the benzamidine.

<center>
<img src="static/bound.png" alt="bound benzamidine" width="600"/>
</center>

## Standard Usage

In [1]:
import os
from main import Arguments, run

### Most important arguments
| Argument    | Type     | Default |
|-------------|:---------|--------:|
| lig         | string   |     LIG |
| top         | string   |  "find" |
| cutoff      | float(Å) |     3.5 |
| maxdist     | float(Å) |     9.0 |

We will set the ligand name to 'BEN', and in order to read the `dcd` trajectories, we will use the file `topology_clean.pdb`.

We have to fetch the example.


In [ ]:
os.chdir("example")

We have a control over what is considered a contact (cutoff), and what is the point when we surely do not have to bias it anymore for it to remain broken (maxdist). We found the defaults fairly universal, but there is a freedom to adapt them to your needs.

You may notice there is a third criterium regarding selecting a distance: it's variance. Biasing a largely flexible contact may distort the structure instead of helping the unbinding. However, this issue primarily emerges when no chemical moiety clustering is employed.

### Clustering
We use a structure based atom grouping, or clustering to deal with molecular symmetries. Take a look at the figure above, and the contact of the benzamidine to the Asp189. Considering the Lewis structure (which manifests in the atom names, not the parameters), there are rotations the system should be invariant to:

<center>
<img src="static/clustering_example.png" alt="clustering" width="400"/>
</center>

Therefore, by default, contacts are biased between the centre of mass of such groups. For protein residues, the clustering is embedded in the code, for ligands, you can define it `toppar/LIG_clusters.dat`. (You will see the three heavy atoms of the amidine group in one line.)

In [22]:
with open("toppar/LIG_clusters.dat", "r") as f:
    for l in f: print(l.strip())

N1 N2 C7


Now we are ready to analyse the unbiased run `traj_0`

In [26]:
args = Arguments(lig="BEN", top="topology_clean.pdb", processonly=True)
run(args)
# processonly is necessary to supress writing inputs for the next iteration


TRAJECTORY 1

Pairs used in this cycle:
3227 5 C7 2486 190 O
3227 5 C7 2471 189 CG
3227 5 C7 2483 190 OG
3227 5 C7 2791 214 O
3227 5 C7 2903 222 CA



The distances are displayed between certain atoms, but in fact, they are groups in the colvar function used in NAMD. Furthermore, they are not biased individually, but their sum.

<center>
<img src="static/bias1.png" alt="initial contacts" width="600"/>
</center>

This now have written a binary checkpoint file in you `example` directory. It is automatically used in further iterations.

In [28]:
args = Arguments(processonly=True)
for _ in range(3):
    run(args)

SyntaxError: invalid syntax (187087062.py, line 1)

After processing `traj_3`, you notice two new distances.
<center>
<img src="static/bias4.png" alt="first new contact" width="600"/>
</center>

The colvars are recorded during the iterations, and they are summerised in `tracked_distances.csv`.

In [34]:
with open("distances_tracked.csv", "r") as f:
    for l in f: print(l.strip())

pairs:     01  02  03  04  05  06  07  08
Traj_01     X   X   X   X   X   0   0   0
Traj_02     X   X   X   X   X   0   0   0
Traj_03     X   X   X   X   X   0   0   0
Traj_04     X   X   X   X   X   X   X   0
Traj_05     X   0   X   X   X   X   X   X

LEGEND
Ligand             Protein
ID  index  resid  type  index  resid  type
01   3227      6  C7     2486    191  O
02   3227      6  C7     2471    190  CG
03   3227      6  C7     2483    191  OG
04   3227      6  C7     2791    215  O
05   3227      6  C7     2903    223  CA
06   3227      6  C7     2799    216  SG
07   3227      6  C7     2784    214  O
08   3227      6  C7     2773    213  O


In [33]:
run(args)   # trajectory 5 is coming up

Distance with id 02 is excluded since the the avarage distance
                        in the last quarter ot the simulation is 9.43 A
TRAJECTORY 5

Pairs used in this cycle:
3227 5 C7 2773 212 O
3227 5 C7 2791 214 O
3227 5 C7 2799 215 SG
3227 5 C7 2784 213 O
3227 5 C7 2486 190 O
3227 5 C7 2483 190 OG
3227 5 C7 2903 222 CA



Now we can see the first distance being excluded.

In [35]:
for _ in range(2): run(args)

Distance with id 03 is excluded since the the avarage distance
                        in the last quarter ot the simulation is 10.74 A
TRAJECTORY 6

Pairs used in this cycle:
3227 5 C7 2507 192 CD
3227 5 C7 2773 212 O
3227 5 C7 2791 214 O
3226 5 C6 2791 214 O
3227 5 C7 2799 215 SG
3227 5 C7 2784 213 O
3227 5 C7 2486 190 O
3227 5 C7 2903 222 CA

TRAJECTORY 7

Pairs used in this cycle:
3227 5 C7 2773 212 O
3227 5 C7 2507 192 CD
3227 5 C7 2791 214 O
3226 5 C6 2791 214 O
3227 5 C7 2799 215 SG
3227 5 C7 2784 213 O
3227 5 C7 2486 190 O
3227 5 C7 2903 222 CA



The process goes on similarly, sometimes adding, sometimes removing contacts from the colvar. At any point, you may see what the status is according to the checkpoint (the interesting parts are on the top, it finishes with the NAMD input):

In [36]:
run(Arguments(report=True))

####      PROGRESS      ####
Current iteration                                                              7
Identified contacts                                                           10
####     UNBINDING      ####
Distance inclusion cutoff                                                   3.50
Distance exclusion cutoff                                                   0.90
Distance fluctuation cutogg                                                 1.00
Trajectory length / ns                                                        10
####       LIGAND       ####
Ligand residue name                                                          BEN
Ligand clusters             
[{'C7', 'N1', 'N2'}]
####    FILE CONFIG     ####
Working directory           /home/berta/PycharmProjects/enhanced-sampling-workshop-2022/Day1/3.Unbinding_Procedure/example
Checkpoint file             /home/berta/PycharmProjects/enhanced-sampling-workshop-2022/Day1/3.Unbinding_Procedure/example/.checkpoint
Topology 

In [37]:
for _ in range(4): run(args)

Distance with id 05 is excluded since the the avarage distance
                        in the last quarter ot the simulation is 9.88 A
TRAJECTORY 8

Pairs used in this cycle:
3227 5 C7 2507 192 CD
3227 5 C7 2773 212 O
3227 5 C7 2791 214 O
3226 5 C6 2791 214 O
3227 5 C7 2799 215 SG
3227 5 C7 2784 213 O
3227 5 C7 2486 190 O

Distance with id 07 is excluded since the the avarage distance
                        in the last quarter ot the simulation is 10.75 A
TRAJECTORY 9

Pairs used in this cycle:
3227 5 C7 2507 192 CD
3227 5 C7 2742 210 O
3227 5 C7 2773 212 O
3227 5 C7 2791 214 O
3226 5 C6 2791 214 O
3227 5 C7 2799 215 SG
3227 5 C7 2486 190 O

Distance with id 10 is excluded since the the avarage distance
                        in the last quarter ot the simulation is 10.50 A
Distance with id 06 is excluded since the the avarage distance
                        in the last quarter ot the simulation is 9.28 A
Distance with id 01 is excluded since the the avarage distance
               

We have finished processing the trajectories saved in this example. Let us inspect the summary.

In [39]:
with open("distances_tracked.csv", "r") as f:
    for l in f: print(l.strip())

TRAJECTORY 11

Pairs used in this cycle:
3227 5 C7 2742 210 O
3227 5 C7 559 58 NE2
3227 5 C7 2507 192 CD
3227 5 C7 2773 212 O
3227 5 C7 2791 214 O

pairs:     01  02  03  04  05  06  07  08  09  10  11  12
Traj_01     X   X   X   X   X   0   0   0   0   0   0   0
Traj_02     X   X   X   X   X   0   0   0   0   0   0   0
Traj_03     X   X   X   X   X   0   0   0   0   0   0   0
Traj_04     X   X   X   X   X   X   X   0   0   0   0   0
Traj_05     X   0   X   X   X   X   X   X   0   0   0   0
Traj_06     X   0   0   X   X   X   X   X   X   X   0   0
Traj_07     X   0   0   X   X   X   X   X   X   X   0   0
Traj_08     X   0   0   X   0   X   X   X   X   X   0   0
Traj_09     X   0   0   X   0   X   0   X   X   X   X   0
Traj_10     0   0   0   X   0   0   0   X   X   0   X   X
Traj_11     0   0   0   X   0   0   0   X   X   0   X   X

LEGEND
Ligand             Protein
ID  index  resid  type  index  resid  type
01   3227      6  C7     2486    191  O
02   3227      6  C7     2471    190  

At this point, the ligand is outside the pocket, exposed to the bulk water. It may wonder around the protein, so depending on the unbinding settings, it may take further iterations to eliminate all contacts. Nevertheless, running an unbiased simulation from this point will also likely result in a free-roaming ligand.

<center>
<img src="static/bias11.png" alt="after 11 iterations" width="600"/>
</center>

To better understand the biasing, run another step, but this time without `processonly`.

In [40]:
run(Arguments())

Distance with id 04 is excluded since the the avarage distance
                        in the last quarter ot the simulation is 9.45 A
TRAJECTORY 12

Pairs used in this cycle:
3227 5 C7 2742 210 O
3227 5 C7 559 58 NE2
3227 5 C7 2507 192 CD
3227 5 C7 2773 212 O



This should create the folder `traj_12` with a NAMD input file `traj_12.inp`, based on the template and the colvar file `sum_12.col`. In the latter, you will see the groups defined by indices (same as in VMD) and that the sum of those is being progressively shifted from 32.79 to 36.79.

In [41]:
with open("traj_12/sum_12.col", "r") as f:
    for l in f: print(l.strip())

#Colletive variables
#generated by CoM_colvar.py
Colvarstrajfrequency    1
Colvarsrestartfrequency 5000
colvar {
name sum_1

distance {
componentCoeff 1.0
group1 {atomnumbers 3230 3228 3229 }
group2 {atomnumbers 2743 }
}
distance {
componentCoeff 1.0
group1 {atomnumbers 3230 3228 3229 }
group2 {atomnumbers 560 }
}
distance {
componentCoeff 1.0
group1 {atomnumbers 3230 3228 3229 }
group2 {atomnumbers 2509 2508 2510 }
}
distance {
componentCoeff 1.0
group1 {atomnumbers 3230 3228 3229 }
group2 {atomnumbers 2774 }
}
}

harmonic {
colvars sum_1
centers 32.79
targetCenters 36.79
targetNumSteps 5000000
forceConstant 20
}


## Additional Options

The complete list of options is available in the [public repository of the unbinding method](https://github.com/rostaresearch/unbinding), here we learn about a few more to play with the results.

You can always rerun any existing step without giving up your checkpoint with `nosave`.

In [42]:
args = Arguments(processonly=True, trajectory=7, nosave=True)
run(args)

Distance with id 05 is excluded since the the avarage distance
                        in the last quarter ot the simulation is 9.88 A
TRAJECTORY 8

Pairs used in this cycle:
3227 5 C7 2507 192 CD
3227 5 C7 2773 212 O
3227 5 C7 2791 214 O
3226 5 C6 2791 214 O
3227 5 C7 2799 215 SG
3227 5 C7 2784 213 O
3227 5 C7 2486 190 O



Should you corrupt your checkpoint, there is also an option to catch up from scratch. This is especially useful if you want to play with the contact definition.

In [44]:
# with cumulative, you will always need the initial setup parameters as well, as it does not use the checkpoint
args = Arguments(cutoff=3.3, maxdist=6, lig="BEN", top="topology_clean.pdb", processonly=True, trajectory=5, nosave=True, cumulative=True)
run(args)


# Cumulative reprocessing #
TRAJECTORY 1

Pairs used in this cycle:
3227 5 C7 2486 190 O
3227 5 C7 2471 189 CG
3227 5 C7 2483 190 OG
3227 5 C7 2791 214 O

Distance with id 04 is excluded since the the avarage distance
                        in the last quarter ot the simulation is 7.17 A
TRAJECTORY 2

Pairs used in this cycle:
3227 5 C7 2486 190 O
3227 5 C7 2471 189 CG
3227 5 C7 2483 190 OG
3227 5 C7 2791 214 O

Distance with id 02 is excluded since the the avarage distance
                        in the last quarter ot the simulation is 6.56 A
Distance with id 03 is excluded since the the avarage distance
                        in the last quarter ot the simulation is 6.14 A
TRAJECTORY 3

Pairs used in this cycle:
3227 5 C7 2486 190 O
3227 5 C7 2791 214 O

TRAJECTORY 4

Pairs used in this cycle:
3227 5 C7 2784 213 O
3227 5 C7 2486 190 O
3227 5 C7 2791 214 O

Distance with id 01 is excluded since the the avarage distance
                        in the last quarter ot the simulation 

## Explore the Unbinding

Here I leave two cells for you to try things. Feel free to dig in and ask questions.

In [ ]:
args = Arguments(
    trajectory=5,
    cumulative=True,
    cutoff=3,
    maxdist=5,
    processonly=True,
    nosave=True,
    lig="BEN",
    top="topology_clean.pdb",
)
run(args)

In [45]:
with open("distances_tracked.csv", "r") as f:
    for l in f: print(l.strip())

pairs:     01  02  03  04  05  06
Traj_01     X   X   X   X   0   0
Traj_02     X   X   X   X   0   0
Traj_03     X   0   0   X   0   0
Traj_04     X   0   0   X   X   0
Traj_05     0   0   0   X   X   0
Traj_06     0   0   0   X   0   X

LEGEND
Ligand             Protein
ID  index  resid  type  index  resid  type
01   3227      6  C7     2486    191  O
02   3227      6  C7     2471    190  CG
03   3227      6  C7     2483    191  OG
04   3227      6  C7     2791    215  O
05   3227      6  C7     2784    214  O
06   3227      6  C7     2507    193  CD
